#### Using data from okxe.vn

In [ ]:
from urllib.request import urlopen
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
url = "https://webscraper.io/test-sites/tables"

In [ ]:
html_code = urlopen(url).read().decode("utf-8")
soup = BeautifulSoup(html_code)

In [ ]:
soup.find_all("h2")

-----------------------------

In [ ]:
brands_url = "https://api.okxe.vn/api/v2/brands?is_active=1"
models_url = "https://api.okxe.vn/api/v2/models?brand_id=1&is_active=1"
posts_url = "https://api.okxe.vn/api/v2/products?sort_of=product_new&page=1&per_page=256"
colors_url = "https://api.okxe.vn/api/v2/colors"

In [ ]:
brands_data = requests.get(brands_url)
posts_data = requests.get(posts_url)
models_data = requests.get(models_url)
colors_data = requests.get(colors_url)

In [ ]:
posts_data.json()

In [ ]:
brands_data.json()['data']

In [ ]:
models_data.json()['data']

In [ ]:
posts_data.json()['data']

In [ ]:
post = posts_data.json()['data'][0]

In [ ]:
att_Vehicle_name = post['product_name']
att_Odometer = post['used_distance']
att_License_plate = post['license_plate']
att_Cubic_power = post['engine']
att_Manufacture_year = post['release_year']
att_City = post['location']['name']
att_ID_VehicleBrand = post['brand_id']
att_ID_VehicleLineup = post['model']['id']
att_Condition = post['used_status']
att_ID_VehicleType = post['type']
att_ID_Color = post['color'][0]['color_id']

In [ ]:
att_Vehicle_name, att_Odometer, att_License_plate, att_Cubic_power, att_Manufacture_year, att_City, att_ID_VehicleBrand, att_ID_VehicleLineup, att_Condition, att_ID_VehicleType, att_ID_Color

In [ ]:
att_Title = post['q_product_name']
att_Content = post['description']
att_Pricetag = post['price']

att_Image = post['image']['medium']

In [ ]:
att_Title, att_Content, att_Pricetag, att_Image

-----

In [ ]:
brands_table = pd.DataFrame(data=brands_data.json()['data']).drop(labels=['logo_normal', 'sort_num', 'is_active'], axis=1)

In [ ]:
brands_table

In [ ]:
total_data = []
for index, row in brands_table.iterrows():
   url = f"https://api.okxe.vn/api/v2/models?brand_id={row['id']}&is_active=1"
   data = requests.get(url).json()['data']
   for i in data:
      i['brand_id'] = row['id']
      total_data.append(i)

In [ ]:
models_table = pd.DataFrame(data=total_data).drop(columns=['image'])

In [ ]:
models_table

In [ ]:
colors_table = pd.DataFrame(data=colors_data.json()['data'])

In [ ]:
colors_table

In [ ]:
condition_data = [
   {'id': 1,
    'name': 'New'},
   {'id': 2,
    'name': 'Used'}
]
condition_table = pd.DataFrame(data=condition_data, index=None)

In [ ]:
vehicletype_data = [
   {'id': 1,
    'name': 'Scooter'},
   {'id': 2,
    'name': 'Manual'},
   {'id': 3,
    'name': 'Motorbike'},
   {'id': 4,
    'name': 'Electric'}
]
vehicletype_table = pd.DataFrame(data=vehicletype_data, index=None)
vehicletype_table

In [ ]:
num_request = 256
total_post = []
total_vehicleinfo = []

url = f"https://api.okxe.vn/api/v2/products?sort_of=product_new&page=1&per_page={num_request}&include=advertisement"
data = requests.get(url).json()['data']

In [ ]:
for j in range(num_request):
   item = data[j]
   vehicle_info = {
      'infoid': j + 1,
      'Vehicle_name': item['product_name'],
      'Odometer': item['used_distance'],
      'License_plate': item['license_plate'],
      'Cubic_power': item['engine'],
      'Manufacture_year': item['release_year'],
      'ID_VehicleBrand': item['brand_id'],
      'ID_VehicleLineup': item['model']['id'],
      'Condition': item['used_status'],
      'ID_VehicleType': item['type'],
      'ID_Color': item['color'][0]['color_id']
   }
   post = {
      'Title': item['q_product_name'],
      'Content': item['description'],
      'Pricetag': item['price'],
      'Image': item['image']['medium'],
      'City_name': item['location']['name'],
      'ID_VehicleInfo': j + 1
   }
   print(item['product_name'] or '###' + " - " + item['license_plate'] or '###')
   total_vehicleinfo.append(vehicle_info)
   total_post.append(post)

In [ ]:
total_vehicleinfo

In [ ]:
posts_table = pd.DataFrame(data=total_post)
vehicleinfo_table = pd.DataFrame(data=total_vehicleinfo)

In [ ]:
posts_table

In [ ]:
posts_table['City_name'].unique(), vehicleinfo_table['Condition'].unique()

In [ ]:
posts_table.isnull().sum()

In [ ]:
mask_new = vehicleinfo_table.Condition == 'new'
mask_used = vehicleinfo_table.Condition == 'used'
vehicleinfo_table.loc[mask_new, 'Condition'] = 1
vehicleinfo_table.loc[mask_used, 'Condition'] = 2

In [ ]:
vehicleinfo_table

In [ ]:
vehicleinfo_table.isnull().sum()

In [ ]:
brands_table.to_csv("VehicleBrand.csv", index=None)
models_table.to_csv("VehicleLineup.csv", index=None)
posts_table.to_csv("Post.csv", index=None)
vehicleinfo_table.to_csv("VehicleInfo.csv", index=None)
colors_table.to_csv('Color.csv', index=None)
condition_table.to_csv('VehicleCondition.csv', index=None)
vehicletype_table.to_csv('VehicleType.csv', index=None)